# pyUmbral Python API

## Setting the default curve

The first time you use umbral, you may want to specify an elliptic curve to use.  If you do not specify a curve, secp256k1 will be used for all operations, with a slight performace hit for the lookup.

To set the default curve use `umbral.config.set_default_curve()`

Note: you can only set the dafault once, or `UmbralConfigurationError` will be raised.

In [1]:
from umbral.config import set_default_curve

set_default_curve()


## Generate Umbral Keys for Alice
First, Let's generate two asymmetric key pairs for Alice:
A *delegating* key pair and a *signing* key pair.


In [2]:
from umbral import keys, signing


# Alice's Keys
alices_private_key = keys.UmbralPrivateKey.gen_key()
alices_public_key = alices_private_key.get_pubkey()

alices_signing_key = keys.UmbralPrivateKey.gen_key()
alices_verifying_key = alices_signing_key.get_pubkey()
alices_signer = signing.Signer(private_key=alices_signing_key)

## Encrypt some data for Alice
Now let's encrypt data with Alice's public key. Invocation of `pre.encrypt` returns both the `ciphertext`,
and a `capsule`. Anyone with Alice's public key can perform this operation.

In [3]:
from umbral import pre


plaintext = b'Proxy Re-encryption is cool!'
ciphertext, capsule = pre.encrypt(alices_public_key, plaintext)
print(ciphertext)


b'\xf7\xc6>/\xe2\xe8L@d\x88\x18/ITvz\xb8\x11\xc7#|\xc3F\x0c$`\x9aj?\x02X\xacz\xbd\x0f>(\xa7\xa3\xad\x18\xb4\xf4\xc3\xc4s\x96ZDN$ \x0e\xf7\xc9$'


## Alice decrypts data for self
Since data was encrypted with Alice's public key, Alice can open the capsule and decrypt the ciphertext with her private key.

In [4]:
cleartext = pre.decrypt(ciphertext=ciphertext, 
						capsule=capsule, 
						decrypting_key=alices_private_key)
print(cleartext)


b'Proxy Re-encryption is cool!'


## Enter Bob
Apart from generating his keypair, we will also assume that Bob receives a capsule through a side channel (s3, ipfs, Google cloud, etc). 

In [5]:
bobs_private_key = keys.UmbralPrivateKey.gen_key()
bobs_public_key = bobs_private_key.get_pubkey()

bob_capsule = capsule

## Attempt Bob's decryption (fail)

In [6]:
try:
    fail_decrypted_data = pre.decrypt(ciphertext=ciphertext, 
    								  capsule=capsule, 
    								  decrypting_key=bobs_private_key)
except:
    print("Decryption failed! Bob doesn't has access granted yet.")


Decryption failed! Bob doesn't has access granted yet.


# Proxy Re-encryption

![proxy_reencryption](https://cdn-images-1.medium.com/max/1200/0*yTKUeeuKPu-aIZdw.)

## Alice grants access to Bob by generating kfrags 
When Alice wants to grant Bob access to open her encrypted messages, she creates *threshold split re-encryption keys*, or *"kfrags"*, which are next sent to N proxies or *Ursulas*. She uses her private key, and Bob's public key, and she sets a minimum threshold of 10, for 20 total shares


In [7]:
M, N = 10, 20
kfrags = pre.split_rekey(delegating_privkey=alices_private_key, 
                         receiving_pubkey=bobs_public_key, 
                         signer=alices_signer,
                         threshold=M, 
                         N=N)



## Ursulas Re-encrypt; Bob attaches fragments to capsule
Bob asks several Ursulas to re-encrypt the capsule so he can open it. Each Ursula performs re-encryption on the capsule using the `kfrag` provided by Alice, obtaining this way a "capsule fragment", or `cfrag`. Let's mock a network or transport layer by sampling `M` random `kfrags`, one for each required Ursula. Bob collects the resulting `cfrags` from several Ursulas. He must gather at least `M` `cfrags` in order to activate the capsule.


In [8]:
import random
kfrags = random.sample(kfrags,  # All kfrags from above
                       10)      # M - Threshold

cfrags = list()                 # Bob's cfrag collection
for kfrag in kfrags:
	cfrag = pre.reencrypt(kfrag=kfrag, capsule=bob_capsule)
	cfrags.append(cfrag)        # Bob collects a cfrag

assert len(cfrags) == 10


## Bob activates and opens the capsule; Decrypts data from Alice.
Bob attaches at least `M` `cfrags` to the capsule. Then it can become *activated*. Finally, Bob activates and opens the capsule, then decrypts the re-encrypted ciphertext.

In [9]:
bob_capsule.set_correctness_keys(delegating=alices_public_key,
                                 receiving=bobs_public_key,
                                 verifying=alices_verifying_key)

for cfrag in cfrags:
	bob_capsule.attach_cfrag(cfrag)
    
bob_cleartext = pre.decrypt(ciphertext=ciphertext, capsule=capsule, decrypting_key=bobs_private_key)
print(bob_cleartext)
assert bob_cleartext == plaintext



b'Proxy Re-encryption is cool!'
